# Proof of Cook Levin Theorem: SAT is NP-complete

The goal of this proof is to show that any abitrary NP computation can be converted to a satisfiability problem (SAT).

### Note

The Turing Machine(TM) implemented here is an Deterministic Turing Machine(DTM) instead of a Non-Deterministic Turing Machine (NDTM). We do this because a DTM is more intuitive to understand. 

However, this doesn't undermine the proof because any problem solvable by NDTM can be solved by DTM and you could also see the computation of the DTM as a path of a NDTM

### Steps

- The `DTM` takes in an arbitrary problem, computes it and returns the computations. This computation is 2-dimensional array of state of the TM at every step of the computation. This array is called a `tableau`
- Using the `Cook_Levin_Prover`, we convert each symbol in the array to a boolean variable with true values in the form.$$x_{i,j,s}$$
where
    - $i$ is the row
    - $j$ is the column
    - $s$ is the value in that cell.

- Then, these four checks are done on these variables and ANDed together:
    1. We check that every entry (i, j) in the computation has exactly one value. Therefore, we check that each entry has one or more values and that each entry cannot have 2 values.
    2. We check that the start state is in the first position of the first row `and` other symbols.
    3. We check that an `ACCEPT_STATE` is in the tableau.
    4. We check that each row yields the next one.
    
### Resources

- [Cook Levin Theorem Proof](https://www.youtube.com/watch?v=LW_37i96htQ)
- [Cook Levin](https://www.cs.ubc.ca/~condon/cpsc506/handouts/Cook-Levin.pdf)

In [5]:
from typing import List, Dict, Optional, Tuple, NewType
from enum import Enum, auto
from functools import reduce

BLANK = "BLANK"

ACCEPT_STATE = "ACCEPT_STATE"

class DIRECTION(Enum):
    LEFT = auto()
    RIGHT = auto()
    
class STATE(Enum):
    ACCEPT = auto()
    REJECT = auto()
    RUNNING = auto()
    

Transition = NewType("Transition", Tuple[str, Optional[str], DIRECTION])

Transitions = NewType("Transitions", Dict[Tuple[str, str], Transition])

def pretty_print(computations):
    for step, config in enumerate(computations):
        print(f"Computation {step}: {config}")
        
class DTM:
    
    state = STATE.RUNNING
    
    head = []
            
    config: List[str] = []
    
    transitions: Transitions = {}
    
    computations: List[Dict] = []
        
    def __init__(self, limit: int, start_state: str, start_config: List[str], transitions: Transitions):
        start_config.insert(0, start_state)
        start_config = [BLANK for _ in range(limit)] + start_config + [BLANK for _ in range(limit)]
        self.config = start_config
        self.head = [start_state, limit]
        self.transitions = transitions
        self.computations = [start_config.copy()]
    
    @staticmethod
    def move(instruction: Tuple[Optional[str], str,  DIRECTION], config: List[str], head: List) -> Tuple[List[str], List]:
        config = config.copy()
        head = head.copy()
        index = head[1]
        (new_state, new_value, direction) = instruction
        if new_value == None:
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                head[0] = new_state
                head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                head[0] = new_state
                head[1] = new_head_index
        else:
            config[head[1]] = new_value
            if direction == DIRECTION.LEFT:
                new_head_index = index - 1
                head[0] = new_state
                head[1] = new_head_index
            elif direction == DIRECTION.RIGHT:
                new_head_index = index + 1
                head[0] = new_state
                head[1] = new_head_index
        return config, head
    
    @staticmethod
    def transition(transitions: Transitions, head: Tuple[str, int], config: List[str], computations: List[List[str]]):
        state = head[0]
        index = head[1]
        config.remove(state)
        symbol = config[index]
                                        
        move = transitions.get((state, symbol))
        
        if move == None:
            state = STATE.REJECT
            return state, head, config, computations
        elif move[0] == ACCEPT_STATE:
            state = STATE.ACCEPT
            new_state = move[0]
            new_value = move[1]
            direction = move[2]
            (config, head) = DTM.move((new_state, new_value, direction), config, head)
            state_index = head[1] - 1
            config.insert(state_index + 1, new_state)
            computations.append(config.copy())
            return state, head, config, computations
        
        new_state = move[0]
        new_value = move[1]
        direction = move[2]
        (config, head) = DTM.move((new_state, new_value, direction), config, head)
        state = STATE.RUNNING
        
        state_index = head[1] - 1
        config.insert(state_index + 1, new_state)
        computations.append(config.copy())
        return state, head, config, computations
                
            
    def run(self):
        (state, head, config, computations) = self.transition(self.transitions, self.head, self.config, self.computations)
        while state == STATE.RUNNING:
            (state, head, config, computations) = self.transition(self.transitions, head, config, computations)
        self.state = state
        self.head = head
        self.config = config
            
start_state = "q0"
start_config = ["0", "0", "0", "0"]
transitions = {
    ("q0", "0"): ("q1", BLANK, DIRECTION.RIGHT),
    ("q1", "x"): ("q1", None, DIRECTION.RIGHT),
    ("q1", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", "x"): ("q2", None, DIRECTION.RIGHT),
    ("q2", "0"): ("q3", None, DIRECTION.RIGHT),
    ("q3", "0"): ("q2", "x", DIRECTION.RIGHT),
    ("q2", BLANK): ("q4", None, DIRECTION.LEFT),
    ("q4", BLANK): ("q1", None, DIRECTION.RIGHT),
    ("q4", "x"): ("q4", None, DIRECTION.LEFT),
    ("q4", "0"): ("q4", None, DIRECTION.LEFT),
    ("q1", BLANK): (ACCEPT_STATE, None, DIRECTION.RIGHT),
}
limit = 2
tm = DTM(limit, start_state, start_config, transitions)
tm.run()
# pretty_print(tm.computations)

### Computation TO SAT

def bool_or(x: bool, y: bool):
    return x | y

def neg_bool_or(x: bool, y: bool):
    return (not x) | (not y)

def bool_and(x: bool, y: bool):
    return x & y

class Cook_Levin_Prover:
    
    def __init__(self, limit: int, computations: List[List[str]], symbols: List[str], states: List[str]):
        self.limit = limit
        self.computations = computations
        self.symbols = symbols
        self.states = states

    @staticmethod
    def computations_to_boolean_vars(computations: List[List[str]]) -> List[List[str]]:
        """
        Converts a set of computations to a booleans variables of true values
        """
        boolean_vars = []
        boolean_values = {}

        for i, computation in enumerate(computations):
            b_vars = [] 
            for j, symbol in enumerate(computation):
                name = f"x_{i}_{j}_{symbol}"
                b_vars.append(name)
                boolean_values[name] = True
            boolean_vars.append(b_vars)
        
        return boolean_vars, boolean_values
    
    @staticmethod
    def condition_1(symbols: List[str], boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that every entry (i, j) in the computation
        has exactly one value. Therefore, we check that each
        entry has one or more values and that each entry cannot
        have 2 values.
        """
        
        res = True
        for i, x in enumerate(boolean_vars):
            for j, y in enumerate(x):
                list_0 = []
                list_1 = []
                for s in symbols:
                    key_1 = f"x_{i}_{j}_{s}"
                    value_1 = (True if boolean_values.get(key_1) == True else False)
                    list_0.append(value_1)
                    for s in symbols:
                        key_2 = f"x_{i}_{j}_{s}"
                        if key_1 == key_2:
                            continue
                        value_2 = True if boolean_values.get(key_2) == True else False
                        list_1.append(neg_bool_or(value_1, value_2))
                        
                s_0 = reduce(bool_or, list_0)
                s_1 = reduce(bool_and, list_1)
                s_2 = bool_and(s_0, s_1)
                res = bool_and(res, s_2)
                
        return res
    
    @staticmethod
    def condition_2(limit:int, boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that the start state is in the first position of the first row `and` other symbols
        """
        start_state = f"x_{0}_{limit}_q0"
        first_row = [boolean_values.get(var) for var in boolean_vars[0]]
        return reduce(bool_and, first_row) and (start_state == boolean_vars[0][limit])
    
    @staticmethod
    def condition_3(boolean_vars: List[List[str]], boolean_values: Dict[str, bool]) -> bool:
        """
        We check that an `ACCEPT_STATE` is in the tableau
        
        This is achieved by looking for an `ACCEPT_STATE` across all the cells(`boolean_vars[i][j]`)
        of the tableau and "ORing" them together.
        
        False is returned if not found.
        """
        accept_states = [boolean_values.get(var) for row in boolean_vars for var in row if ACCEPT_STATE in var]
        ## This serves as an identity element for when there are not accept states
        accept_states.append(False)
        return reduce(bool_or, accept_states)
    
    @staticmethod
    def condition_4(computations: List[List[str]], states: List[str]) -> bool:
        """
        We check that each row is a transition of the previous row
        """
        
        def get_state_and_index(computation: List[str], states: List[str]):
            for index, symbol in enumerate(computation):
                if symbol in states:
                    return index, symbol
                
        def get_windows(state_index: int, current_row: List[str], next_row: List[str]) -> Tuple[List[List[str]], List[List[str]], List[List[str]]]:
            before = state_index - 1
            after = state_index + 1
            left_window = [current_row[:before], next_row[:before]]
            middle_window = [current_row[before:after + 1], next_row[before:after + 1]]
            right_window = [current_row[after + 1:], next_row[after + 1:]]
            return (left_window, middle_window, right_window)
        
        boolean_values = []
        
        for row_index in range(len(computations)):
            if row_index == (len(computations) - 1):
                continue
            current_row = computations[row_index]
            next_row = computations[row_index + 1]
            state_in_current_row, index = get_state_and_index(current_row, states)
            (lw, mw, rw) = get_windows(state_in_current_row, current_row, next_row)
            boolean_values.append(lw[0] == lw[1])
            boolean_values.append(rw[0] == rw[1])
            boolean_values.append(mw[0] != mw[1])
            
        return reduce(bool_and, boolean_values)
    
            
    def run(self):
        boolean_vars, boolean_values = self.computations_to_boolean_vars(self.computations)
        condition_1 = self.condition_1(self.symbols, boolean_vars, boolean_values)
        condition_2 = self.condition_2(self.limit, boolean_vars, boolean_values)
        condition_3 = self.condition_3(boolean_vars, boolean_values)
        condition_4 = self.condition_4(self.computations, self.states)
        final_conditions = [condition_1, condition_2, condition_3, condition_4]
        return reduce(bool_and, final_conditions)


symbols = ["q0", "q1", "q2", "q3", "q4", "0", "x", BLANK, ACCEPT_STATE]
states = ["q0", "q1", "q2", "q3", "q4", ACCEPT_STATE]
cook_levin_prover = Cook_Levin_Prover(limit, tm.computations, symbols, states)
cook_levin_prover.run()

True

# Freivalds Algorithm

In [8]:
"""
- The product of two matrices is equalavent to performing two transformation

- FORMULAR FOR PRODUCT OF TWO n x n MATRICES

- The order matters

 M2 = [[a, b],[c, d]]
 
 M1 = [[e, f],[g, h]]
 
 M2 * [e, g] = e * [a, c] + g * [b, d] = [(a * e) + (b * g), (c * e) + (d * g)]
 
 M2 * [f, h] = f * [a, c] + h * [b, d] = [(a * f) + (b * h), (c * f) + (d * h)]
 
 M2 * M1 = [[((a * e) + (b * g)), ((a * f) + (b * h))],[((c * e) + (d * g)), ((c * f) + (d * h))]]
"""

from typing import List, Tuple
import numpy as np

# for i in range(3):
#     for j in range(3):
#         print(j, i)
# This shows that how to index by (column, row) rather than (row, column)

def scalar_mul(a: int, b: List[int]):
    res = []
    for value in b:
        res.append(a * value)
    return res

def combine(a: List[int], b: List[int]):
    if len(a) == 0 and len(b) != 0:
        return b
    elif len(b) == 0 and len(a) != 0:
        return a
    
    assert len(a) == len(b)
    
    res = []
    for i, j in zip(a, b):
        res.append(i + j)
    return res

def linear_combination(a: List[int], b: List[List[int]]):
    assert len(a) == len(b)
    n = len(a)
    res = []
    for i in range(n):
        scalar = a[i]
        vector = b[i]
        mul = scalar_mul(scalar, vector)
        res = combine(res, mul)
    return res
        
    
def matrix_product(A, B):
    n = len(A)
    # Initialize an n x n matrix with zeros for the result
    C = [[0] * n for _ in range(n)]
    
    # Perform matrix multiplication
    for i in range(n):
        for j in range(n):
            for k in range(n):
                C[i][j] += A[i][k] * B[k][j]
    
    return C

############################
### Freivald's Algorithm ###
############################

"""
First,choose a random r ∈ Fp,and let x = (1, r, r2,...,rn−1). Then compute y = Cx and z = A·Bx, outputting YES if y = z and NO otherwise.
"""

import random

p = 79

r = random.randint(5, p - 1)

A = [[0, 2], [1, 0]]

B = [[1, -2], [1, 0]]

x = [r ** i for i in range(len(A))]

C = matrix_product(A, B)

print(x)

Bx = linear_combination(x, B)

print(Bx)

z = linear_combination(Bx, A)

print(z)

y = linear_combination(x, C)

print(y)

# assert z == y

[1, 48]
[49, -2]
[-2, 98]
[50, -96]


# Showing that finding a path in a Directed Graph can be gotten in Polynomial Time

In [11]:
from typing import List

def get_nodes(edges: List[List[str]]):
    nodes = []

    for edge in edges:

        if edge[0] not in nodes:
            nodes.append(edge[0])

        if edge[1] not in nodes:
            nodes.append(edge[1])
            
    nodes.sort()
    return nodes


def checker(edges: List[List[str]], start: str, end: str) -> bool:
    
    marked = {start: True}

    for edge in edges: 
        if marked.get(edge[0]):
            marked[edge[1]] = True
            
    return True if marked.get(end) is not None else False


## Check that a path exists from a to f

edges = [["a", "b"], ["a", "c"], ["b", "e"], ["e", "c"], ["d", "f"]]
print(checker(edges, "a", "e"))

True
